# Table Creation and DML Operations

## Learning Objectives
By the end of this notebook, you will be able to:
- Create database tables with proper data types and constraints
- Perform DML operations (INSERT, UPDATE, DELETE, SELECT)
- Use parameterized queries for security and performance
- Handle transactions and data integrity
- Work with indexes and performance optimization

## 1. Table Creation (DDL Operations)

In [ ]:
import sqlite3
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime, date
from contextlib import contextmanager

@contextmanager
def get_db_connection(db_path: str = "company_database.db"):
    """
    Context manager for database connections.
    """
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row
    try:
        yield conn
    finally:
        conn.close()

# Create comprehensive database schema
def create_company_schema() -> bool:
    """
    Create a complete company database schema.
    
    Returns:
        True if successful, False otherwise
    """
    
    # SQL statements for table creation
    create_tables_sql = [
        # Departments table
        """
        CREATE TABLE IF NOT EXISTS departments (
            department_id INTEGER PRIMARY KEY AUTOINCREMENT,
            department_name VARCHAR(100) NOT NULL UNIQUE,
            manager_id INTEGER,
            budget DECIMAL(15, 2) DEFAULT 0.00,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            is_active BOOLEAN DEFAULT 1
        )
        """,
        
        # Employees table
        """
        CREATE TABLE IF NOT EXISTS employees (
            employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
            employee_code VARCHAR(20) NOT NULL UNIQUE,
            first_name VARCHAR(50) NOT NULL,
            last_name VARCHAR(50) NOT NULL,
            email VARCHAR(100) UNIQUE NOT NULL,
            phone VARCHAR(20),
            hire_date DATE NOT NULL,
            department_id INTEGER,
            position VARCHAR(100),
            salary DECIMAL(10, 2) NOT NULL,
            is_active BOOLEAN DEFAULT 1,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (department_id) REFERENCES departments(department_id)
        )
        """,
        
        # Projects table
        """
        CREATE TABLE IF NOT EXISTS projects (
            project_id INTEGER PRIMARY KEY AUTOINCREMENT,
            project_name VARCHAR(200) NOT NULL,
            description TEXT,
            start_date DATE NOT NULL,
            end_date DATE,
            budget DECIMAL(15, 2),
            status VARCHAR(20) DEFAULT 'PLANNING',
            department_id INTEGER,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (department_id) REFERENCES departments(department_id),
            CHECK (status IN ('PLANNING', 'ACTIVE', 'COMPLETED', 'CANCELLED'))
        )
        """,
        
        # Employee Projects (Many-to-Many relationship)
        """
        CREATE TABLE IF NOT EXISTS employee_projects (
            employee_id INTEGER,
            project_id INTEGER,
            role VARCHAR(50),
            allocation_percentage DECIMAL(5, 2) DEFAULT 100.00,
            start_date DATE NOT NULL,
            end_date DATE,
            PRIMARY KEY (employee_id, project_id),
            FOREIGN KEY (employee_id) REFERENCES employees(employee_id),
            FOREIGN KEY (project_id) REFERENCES projects(project_id),
            CHECK (allocation_percentage > 0 AND allocation_percentage <= 100)
        )
        """,
        
        # Salary History table
        """
        CREATE TABLE IF NOT EXISTS salary_history (
            history_id INTEGER PRIMARY KEY AUTOINCREMENT,
            employee_id INTEGER NOT NULL,
            old_salary DECIMAL(10, 2),
            new_salary DECIMAL(10, 2) NOT NULL,
            change_date DATE NOT NULL,
            change_reason VARCHAR(200),
            created_by VARCHAR(50),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
        )
        """
    ]
    
    try:
        with get_db_connection() as conn:
            # Enable foreign key constraints
            conn.execute("PRAGMA foreign_keys = ON")
            
            # Create all tables
            for sql in create_tables_sql:
                conn.execute(sql)
            
            conn.commit()
            print("✓ All tables created successfully")
            return True
            
    except sqlite3.Error as e:
        print(f"Error creating tables: {e}")
        return False

# Create the database schema
success = create_company_schema()
if success:
    print("Database schema created successfully!")
else:
    print("Failed to create database schema")

In [ ]:
# Create indexes for better performance
def create_indexes() -> bool:
    """
    Create indexes to improve query performance.
    
    Returns:
        True if successful, False otherwise
    """
    
    index_statements = [
        "CREATE INDEX IF NOT EXISTS idx_employees_department ON employees(department_id)",
        "CREATE INDEX IF NOT EXISTS idx_employees_email ON employees(email)",
        "CREATE INDEX IF NOT EXISTS idx_employees_hire_date ON employees(hire_date)",
        "CREATE INDEX IF NOT EXISTS idx_projects_status ON projects(status)",
        "CREATE INDEX IF NOT EXISTS idx_projects_dates ON projects(start_date, end_date)",
        "CREATE INDEX IF NOT EXISTS idx_salary_history_employee ON salary_history(employee_id)",
        "CREATE INDEX IF NOT EXISTS idx_salary_history_date ON salary_history(change_date)"
    ]
    
    try:
        with get_db_connection() as conn:
            for index_sql in index_statements:
                conn.execute(index_sql)
            
            conn.commit()
            print("✓ All indexes created successfully")
            return True
            
    except sqlite3.Error as e:
        print(f"Error creating indexes: {e}")
        return False

# Create indexes
create_indexes()

# Verify table creation
def show_table_info() -> None:
    """
    Display information about created tables.
    """
    try:
        with get_db_connection() as conn:
            # Get list of tables
            cursor = conn.execute("""
                SELECT name FROM sqlite_master 
                WHERE type='table' AND name NOT LIKE 'sqlite_%'
                ORDER BY name
            """)
            
            tables = cursor.fetchall()
            print(f"\nCreated {len(tables)} tables:")
            
            for table in tables:
                table_name = table['name']
                print(f"\n📋 Table: {table_name}")
                
                # Get column information
                cursor = conn.execute(f"PRAGMA table_info({table_name})")
                columns = cursor.fetchall()
                
                for col in columns:
                    pk_indicator = " (PK)" if col['pk'] else ""
                    not_null = " NOT NULL" if col['notnull'] else ""
                    default = f" DEFAULT {col['dflt_value']}" if col['dflt_value'] else ""
                    print(f"  • {col['name']}: {col['type']}{pk_indicator}{not_null}{default}")
                    
    except sqlite3.Error as e:
        print(f"Error showing table info: {e}")

show_table_info()

## 2. INSERT Operations

In [ ]:
# Insert sample data into tables
def insert_sample_departments() -> bool:
    """
    Insert sample department data.
    
    Returns:
        True if successful, False otherwise
    """
    
    departments_data = [
        ('Engineering', None, 500000.00),
        ('Marketing', None, 200000.00),
        ('Sales', None, 300000.00),
        ('Human Resources', None, 150000.00),
        ('Finance', None, 250000.00)
    ]
    
    try:
        with get_db_connection() as conn:
            # Insert departments
            cursor = conn.executemany(
                "INSERT INTO departments (department_name, manager_id, budget) VALUES (?, ?, ?)",
                departments_data
            )
            
            conn.commit()
            print(f"✓ Inserted {cursor.rowcount} departments")
            return True
            
    except sqlite3.Error as e:
        print(f"Error inserting departments: {e}")
        return False

def insert_sample_employees() -> bool:
    """
    Insert sample employee data.
    
    Returns:
        True if successful, False otherwise
    """
    
    employees_data = [
        ('EMP001', 'Alice', 'Johnson', 'alice.johnson@company.com', '+1-555-0101', '2022-01-15', 1, 'Senior Developer', 85000.00),
        ('EMP002', 'Bob', 'Smith', 'bob.smith@company.com', '+1-555-0102', '2021-06-01', 2, 'Marketing Manager', 75000.00),
        ('EMP003', 'Charlie', 'Brown', 'charlie.brown@company.com', '+1-555-0103', '2023-03-10', 3, 'Sales Representative', 55000.00),
        ('EMP004', 'Diana', 'Davis', 'diana.davis@company.com', '+1-555-0104', '2020-11-20', 1, 'Lead Engineer', 95000.00),
        ('EMP005', 'Eve', 'Wilson', 'eve.wilson@company.com', '+1-555-0105', '2022-08-05', 4, 'HR Specialist', 60000.00),
        ('EMP006', 'Frank', 'Miller', 'frank.miller@company.com', '+1-555-0106', '2021-12-01', 5, 'Financial Analyst', 70000.00),
        ('EMP007', 'Grace', 'Lee', 'grace.lee@company.com', '+1-555-0107', '2023-01-15', 1, 'Junior Developer', 65000.00),
        ('EMP008', 'Henry', 'Clark', 'henry.clark@company.com', '+1-555-0108', '2022-04-10', 2, 'Content Specialist', 50000.00)
    ]
    
    try:
        with get_db_connection() as conn:
            # Insert employees
            cursor = conn.executemany(
                """
                INSERT INTO employees 
                (employee_code, first_name, last_name, email, phone, hire_date, 
                 department_id, position, salary) 
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                employees_data
            )
            
            conn.commit()
            print(f"✓ Inserted {cursor.rowcount} employees")
            return True
            
    except sqlite3.Error as e:
        print(f"Error inserting employees: {e}")
        return False

def insert_sample_projects() -> bool:
    """
    Insert sample project data.
    
    Returns:
        True if successful, False otherwise
    """
    
    projects_data = [
        ('Customer Portal Redesign', 'Redesign the customer-facing web portal', '2024-01-01', '2024-06-30', 150000.00, 'ACTIVE', 1),
        ('Marketing Campaign Q1', 'Q1 2024 marketing campaign for new products', '2024-01-01', '2024-03-31', 75000.00, 'COMPLETED', 2),
        ('Sales Process Automation', 'Automate the sales lead qualification process', '2024-02-01', '2024-08-31', 200000.00, 'ACTIVE', 3),
        ('Employee Training Platform', 'Develop internal training and certification platform', '2024-03-01', '2024-12-31', 100000.00, 'PLANNING', 4),
        ('Financial Reporting System', 'Upgrade financial reporting and analytics system', '2024-01-15', '2024-09-15', 180000.00, 'ACTIVE', 5)
    ]
    
    try:
        with get_db_connection() as conn:
            # Insert projects
            cursor = conn.executemany(
                """
                INSERT INTO projects 
                (project_name, description, start_date, end_date, budget, status, department_id) 
                VALUES (?, ?, ?, ?, ?, ?, ?)
                """,
                projects_data
            )
            
            conn.commit()
            print(f"✓ Inserted {cursor.rowcount} projects")
            return True
            
    except sqlite3.Error as e:
        print(f"Error inserting projects: {e}")
        return False

# Insert all sample data
print("Inserting sample data...")
insert_sample_departments()
insert_sample_employees()
insert_sample_projects()
print("Sample data insertion completed!")

## 3. SELECT Operations

In [ ]:
# Various SELECT operations
def demonstrate_select_operations() -> None:
    """
    Demonstrate various SELECT query patterns.
    """
    
    try:
        with get_db_connection() as conn:
            
            # 1. Basic SELECT with all columns
            print("1. All Departments:")
            cursor = conn.execute("SELECT * FROM departments ORDER BY department_name")
            departments = cursor.fetchall()
            
            for dept in departments:
                print(f"   {dept['department_id']}: {dept['department_name']} (Budget: ${dept['budget']:,.2f})")
            
            # 2. SELECT with specific columns and WHERE clause
            print("\n2. Engineering Department Employees:")
            cursor = conn.execute("""
                SELECT first_name, last_name, position, salary 
                FROM employees 
                WHERE department_id = 1 
                ORDER BY salary DESC
            """)
            
            eng_employees = cursor.fetchall()
            for emp in eng_employees:
                print(f"   {emp['first_name']} {emp['last_name']} - {emp['position']} (${emp['salary']:,.2f})")
            
            # 3. JOIN operations
            print("\n3. Employees with Department Names:")
            cursor = conn.execute("""
                SELECT e.first_name, e.last_name, e.position, d.department_name, e.salary
                FROM employees e
                JOIN departments d ON e.department_id = d.department_id
                ORDER BY e.salary DESC
                LIMIT 5
            """)
            
            top_employees = cursor.fetchall()
            for emp in top_employees:
                print(f"   {emp['first_name']} {emp['last_name']} ({emp['department_name']}) - ${emp['salary']:,.2f}")
            
            # 4. Aggregate functions
            print("\n4. Department Statistics:")
            cursor = conn.execute("""
                SELECT 
                    d.department_name,
                    COUNT(e.employee_id) as employee_count,
                    AVG(e.salary) as avg_salary,
                    MIN(e.salary) as min_salary,
                    MAX(e.salary) as max_salary
                FROM departments d
                LEFT JOIN employees e ON d.department_id = e.department_id
                GROUP BY d.department_id, d.department_name
                ORDER BY employee_count DESC
            """)
            
            dept_stats = cursor.fetchall()
            for stat in dept_stats:
                avg_sal = stat['avg_salary'] if stat['avg_salary'] else 0
                min_sal = stat['min_salary'] if stat['min_salary'] else 0
                max_sal = stat['max_salary'] if stat['max_salary'] else 0
                
                print(f"   {stat['department_name']}: {stat['employee_count']} employees")
                print(f"     Avg: ${avg_sal:,.2f}, Min: ${min_sal:,.2f}, Max: ${max_sal:,.2f}")
            
            # 5. Subqueries
            print("\n5. Employees with Above-Average Salaries:")
            cursor = conn.execute("""
                SELECT first_name, last_name, salary
                FROM employees
                WHERE salary > (SELECT AVG(salary) FROM employees)
                ORDER BY salary DESC
            """)
            
            above_avg = cursor.fetchall()
            for emp in above_avg:
                print(f"   {emp['first_name']} {emp['last_name']}: ${emp['salary']:,.2f}")
            
            # 6. Date-based queries
            print("\n6. Recently Hired Employees (2022 or later):")
            cursor = conn.execute("""
                SELECT first_name, last_name, hire_date, position
                FROM employees
                WHERE hire_date >= '2022-01-01'
                ORDER BY hire_date DESC
            """)
            
            recent_hires = cursor.fetchall()
            for emp in recent_hires:
                print(f"   {emp['first_name']} {emp['last_name']} - {emp['position']} (Hired: {emp['hire_date']})")
                
    except sqlite3.Error as e:
        print(f"Error in SELECT operations: {e}")

demonstrate_select_operations()

## 4. UPDATE Operations

In [ ]:
# UPDATE operations with different scenarios
def demonstrate_update_operations() -> None:
    """
    Demonstrate various UPDATE operations.
    """
    
    try:
        with get_db_connection() as conn:
            
            # 1. Single record update
            print("1. Updating Alice Johnson's salary:")
            
            # First, show current salary
            cursor = conn.execute(
                "SELECT first_name, last_name, salary FROM employees WHERE employee_code = ?",
                ('EMP001',)
            )
            current = cursor.fetchone()
            print(f"   Current: {current['first_name']} {current['last_name']} - ${current['salary']:,.2f}")
            
            # Update salary
            cursor = conn.execute(
                "UPDATE employees SET salary = ?, updated_at = CURRENT_TIMESTAMP WHERE employee_code = ?",
                (90000.00, 'EMP001')
            )
            
            print(f"   Updated {cursor.rowcount} record(s)")
            
            # Show updated salary
            cursor = conn.execute(
                "SELECT first_name, last_name, salary FROM employees WHERE employee_code = ?",
                ('EMP001',)
            )
            updated = cursor.fetchone()
            print(f"   Updated: {updated['first_name']} {updated['last_name']} - ${updated['salary']:,.2f}")
            
            # 2. Bulk update with conditions
            print("\n2. Giving 5% raise to all Engineering employees:")
            
            # Show current Engineering salaries
            cursor = conn.execute("""
                SELECT first_name, last_name, salary 
                FROM employees 
                WHERE department_id = 1
                ORDER BY salary DESC
            """)
            
            eng_before = cursor.fetchall()
            print("   Before raise:")
            for emp in eng_before:
                print(f"     {emp['first_name']} {emp['last_name']}: ${emp['salary']:,.2f}")
            
            # Apply 5% raise
            cursor = conn.execute("""
                UPDATE employees 
                SET salary = salary * 1.05, 
                    updated_at = CURRENT_TIMESTAMP 
                WHERE department_id = 1
            """)
            
            print(f"   Applied raise to {cursor.rowcount} employees")
            
            # Show updated salaries
            cursor = conn.execute("""
                SELECT first_name, last_name, salary 
                FROM employees 
                WHERE department_id = 1
                ORDER BY salary DESC
            """)
            
            eng_after = cursor.fetchall()
            print("   After raise:")
            for emp in eng_after:
                print(f"     {emp['first_name']} {emp['last_name']}: ${emp['salary']:,.2f}")
            
            # 3. Conditional update with JOIN
            print("\n3. Updating project status based on end date:")
            
            # Show current project statuses
            cursor = conn.execute(
                "SELECT project_name, status, end_date FROM projects ORDER BY end_date"
            )
            
            projects_before = cursor.fetchall()
            print("   Current project statuses:")
            for proj in projects_before:
                print(f"     {proj['project_name']}: {proj['status']} (End: {proj['end_date']})")
            
            # Update completed projects (end date in the past)
            cursor = conn.execute("""
                UPDATE projects 
                SET status = 'COMPLETED' 
                WHERE end_date < date('now') AND status != 'COMPLETED'
            """)
            
            print(f"   Updated {cursor.rowcount} project(s) to COMPLETED status")
            
            # 4. Update with calculated values
            print("\n4. Updating department budgets based on employee salaries:")
            
            cursor = conn.execute("""
                UPDATE departments 
                SET budget = (
                    SELECT COALESCE(SUM(salary) * 2, 0) 
                    FROM employees 
                    WHERE employees.department_id = departments.department_id
                )
                WHERE department_id IN (SELECT DISTINCT department_id FROM employees)
            """)
            
            print(f"   Updated {cursor.rowcount} department budget(s)")
            
            # Show updated budgets
            cursor = conn.execute("""
                SELECT department_name, budget 
                FROM departments 
                ORDER BY budget DESC
            """)
            
            budgets = cursor.fetchall()
            print("   Updated department budgets:")
            for dept in budgets:
                print(f"     {dept['department_name']}: ${dept['budget']:,.2f}")
            
            # Commit all changes
            conn.commit()
            print("\n✓ All updates committed successfully")
            
    except sqlite3.Error as e:
        print(f"Error in UPDATE operations: {e}")
        if conn:
            conn.rollback()
            print("Changes rolled back")

demonstrate_update_operations()

## 5. DELETE Operations

In [ ]:
# DELETE operations with safety measures
def demonstrate_delete_operations() -> None:
    """
    Demonstrate various DELETE operations with safety measures.
    """
    
    try:
        with get_db_connection() as conn:
            
            # First, let's add some test data that we can safely delete
            print("1. Adding test data for deletion demonstration:")
            
            # Add a test department
            cursor = conn.execute(
                "INSERT INTO departments (department_name, budget) VALUES (?, ?)",
                ('Test Department', 10000.00)
            )
            test_dept_id = cursor.lastrowid
            
            # Add test employees
            test_employees = [
                ('TEST001', 'Test', 'Employee1', 'test1@company.com', None, '2024-01-01', test_dept_id, 'Test Position', 50000.00),
                ('TEST002', 'Test', 'Employee2', 'test2@company.com', None, '2024-01-01', test_dept_id, 'Test Position', 55000.00)
            ]
            
            cursor = conn.executemany(
                """
                INSERT INTO employees 
                (employee_code, first_name, last_name, email, phone, hire_date, 
                 department_id, position, salary) 
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                test_employees
            )
            
            print(f"   Added test department (ID: {test_dept_id}) and {len(test_employees)} test employees")
            
            # 2. Safe single record deletion
            print("\n2. Deleting a single test employee:")
            
            # First, verify the record exists
            cursor = conn.execute(
                "SELECT employee_id, first_name, last_name FROM employees WHERE employee_code = ?",
                ('TEST001',)
            )
            
            employee_to_delete = cursor.fetchone()
            if employee_to_delete:
                print(f"   Found employee: {employee_to_delete['first_name']} {employee_to_delete['last_name']}")
                
                # Delete the employee
                cursor = conn.execute(
                    "DELETE FROM employees WHERE employee_code = ?",
                    ('TEST001',)
                )
                
                print(f"   Deleted {cursor.rowcount} employee record")
            
            # 3. Conditional deletion with WHERE clause
            print("\n3. Deleting employees with salary below threshold:")
            
            # Show employees with salary < 52000 in test department
            cursor = conn.execute("""
                SELECT employee_code, first_name, last_name, salary 
                FROM employees 
                WHERE department_id = ? AND salary < 52000
            """, (test_dept_id,))
            
            low_salary_employees = cursor.fetchall()
            print(f"   Found {len(low_salary_employees)} employees with salary < $52,000:")
            
            for emp in low_salary_employees:
                print(f"     {emp['employee_code']}: {emp['first_name']} {emp['last_name']} (${emp['salary']:,.2f})")
            
            # Delete them (this is safe since they're test employees)
            cursor = conn.execute(
                "DELETE FROM employees WHERE department_id = ? AND salary < 52000",
                (test_dept_id,)
            )
            
            print(f"   Deleted {cursor.rowcount} employee record(s)")
            
            # 4. Cascading deletion (delete department and remaining employees)
            print("\n4. Cascading deletion - removing test department:")
            
            # First, count remaining employees in test department
            cursor = conn.execute(
                "SELECT COUNT(*) as count FROM employees WHERE department_id = ?",
                (test_dept_id,)
            )
            
            remaining_count = cursor.fetchone()['count']
            print(f"   Remaining employees in test department: {remaining_count}")
            
            # Delete remaining employees first (to avoid foreign key constraint)
            if remaining_count > 0:
                cursor = conn.execute(
                    "DELETE FROM employees WHERE department_id = ?",
                    (test_dept_id,)
                )
                print(f"   Deleted {cursor.rowcount} remaining employee(s)")
            
            # Now delete the department
            cursor = conn.execute(
                "DELETE FROM departments WHERE department_id = ?",
                (test_dept_id,)
            )
            
            print(f"   Deleted {cursor.rowcount} department record")
            
            # 5. Demonstrate DELETE with JOIN (using subquery)
            print("\n5. Advanced deletion using subquery:")
            
            # Let's create a scenario: delete all projects from departments with no employees
            # First, show departments with no employees
            cursor = conn.execute("""
                SELECT d.department_id, d.department_name
                FROM departments d
                LEFT JOIN employees e ON d.department_id = e.department_id
                WHERE e.employee_id IS NULL
            """)
            
            empty_departments = cursor.fetchall()
            print(f"   Found {len(empty_departments)} departments with no employees:")
            
            for dept in empty_departments:
                print(f"     {dept['department_name']} (ID: {dept['department_id']})")
            
            # Count projects in these departments
            if empty_departments:
                empty_dept_ids = [dept['department_id'] for dept in empty_departments]
                placeholders = ','.join('?' * len(empty_dept_ids))
                
                cursor = conn.execute(
                    f"SELECT COUNT(*) as count FROM projects WHERE department_id IN ({placeholders})",
                    empty_dept_ids
                )
                
                project_count = cursor.fetchone()['count']
                print(f"   Projects in empty departments: {project_count}")
                
                # Note: In a real scenario, you might want to reassign these projects
                # rather than delete them. For demonstration, we'll just show the count.
            
            # Commit all changes
            conn.commit()
            print("\n✓ All deletion operations completed successfully")
            
            # Show final counts
            cursor = conn.execute("SELECT COUNT(*) as count FROM departments")
            dept_count = cursor.fetchone()['count']
            
            cursor = conn.execute("SELECT COUNT(*) as count FROM employees")
            emp_count = cursor.fetchone()['count']
            
            cursor = conn.execute("SELECT COUNT(*) as count FROM projects")
            proj_count = cursor.fetchone()['count']
            
            print(f"\nFinal counts: {dept_count} departments, {emp_count} employees, {proj_count} projects")
            
    except sqlite3.Error as e:
        print(f"Error in DELETE operations: {e}")
        if conn:
            conn.rollback()
            print("Changes rolled back")

demonstrate_delete_operations()